In [ ]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow as tf
import pandas as pd
import evaluate
import os
dir_root = os.getcwd()

In [ ]:
from transformers import TFAutoModelForSequenceClassification, TFDistilBertForSequenceClassification
model1 = TFAutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/models/pretrainedModel', num_labels=6)

Some layers from the model checkpoint at /content/drive/MyDrive/models/pretrainedModel were not used when initializing TFDistilBertForSequenceClassification: ['dropout_39']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/models/pretrainedModel and are newly initialized: ['dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model1.compile(optimizer='adam', metrics = ['accuracy'], loss=loss)

In [ ]:
testcsv = os.path.join(dir_root, '/content/drive/MyDrive/data/interim/testsethugf.csv')

In [ ]:
labels_set = {'ROBO A DOMICILIO',
 'ROBO A PERSONAS',
 'ROBO A UNIDADES ECONOMICAS',
 'ROBO DE BIENES, ACCESORIOS Y AUTOPARTES DE VEHICULOS',
 'ROBO DE CARROS',
 'ROBO DE MOTOS'}

In [ ]:
from datasets import load_dataset, Features, ClassLabel, Value

class_names = list(labels_set)
robo_features = Features({'relato': Value('string'), 'labels': ClassLabel(names=class_names)})
dataset = load_dataset("csv", data_files={'test': testcsv, }, features=robo_features)

Using custom data configuration default-2918cfdbd187ae63


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-2918cfdbd187ae63/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
from transformers import DistilBertTokenizerFast
model_name = 'distilbert-base-multilingual-cased'
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)

In [ ]:
def tokenizer_func(examples):
  return tokenizer(examples["relato"],
                   max_length=300,
                   padding = "max_length",
                   truncation=True)

tokenized_dataset = dataset.map(tokenizer_func, batched=True)

Parameter 'function'=<function tokenizer_func at 0x7fdb673bdd40> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/90 [00:00<?, ?ba/s]

In [ ]:
len(tokenized_dataset['test'][-1]['input_ids']), tokenized_dataset['test'][-1]['labels']

(300, 1)

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [ ]:
tf_test_set = tokenized_dataset["test"].shuffle(seed=42).select(range(4000)).to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [ ]:
model1.evaluate(tf_test_set)

170/250 [===================>..........] - ETA: 12:28 - loss: 3.2602 - accuracy: 0.0592

The performance has nothing to do with what was trained. I wonder if I changed the model to distilbert we had the same performance

In [ ]:
model_notrainig = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6)

In [ ]:
model_notrainig.compile(optimizer='adam', metrics = ['accuracy'], loss=loss)
model_notrainig.evaluate(tf_test_set)

IT seems that the real performance of the model is below 0.46

Trying to load the model from weights

In [ ]:
model_2 = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6)
model_2.compile(optimizer='adam', metrics = ['accuracy'], loss=loss)

In [ ]:
model_2.load_weights('/content/drive/MyDrive/models/tfmodelnlph5.h5')

In [ ]:
model_2.summary()

In [ ]:
model_2.evaluate(tf_test_set)